# Group14_Project3

In [3]:
# Sai Madhan Muthyam 
# Karan Raturi
# Lakshmi Vinanya Yenumula 
# Venkata Rama Anirudh Vikram Kesapragada
# Neelima Nandigam
# Amol Bhadane

In [1]:
sc

# Step 2: Data Analysis Using Spark Core

## A. Blockchain Data Analysis – Part 1

In [2]:
import json

file_path = "hdfs://localhost/loudacre/blockchain_data.txt"
json_string = sc.textFile(file_path).reduce(lambda x, y: x + y)
data_blocks = json.loads(json_string)

In [3]:
total_blocks = len(data_blocks)
print("Total number of blocks:", total_blocks)

('Total number of blocks:', 1026)


In [4]:
largest_block_height = max(block['height'] for block in data_blocks)
print("Largest block height:", largest_block_height)

('Largest block height:', 835509)


In [5]:
block_with_largest_height = next((block for block in data_blocks if block.get('height') == largest_block_height), None)

if block_with_largest_height:
    time_largest_block = block_with_largest_height.get('time')
    print("Time for the largest block:", time_largest_block)

('Time for the largest block:', 1710924274)


In [7]:
# Initialize variables to store the highest number of transactions and corresponding block height
max_transactions = 0
highest_transactions_block_height = None

# Iterate over the data to find the block with the highest number of transactions
for block in data_blocks:
    num_transactions = block['n_tx']
    if num_transactions > max_transactions:
        max_transactions = num_transactions
        highest_transactions_block_height = block['height']

print("Block with the highest number of transactions:", highest_transactions_block_height)
print("Highest number of transactions in the blocks:", max_transactions)


('Block with the highest number of transactions:', 834900)
('Highest number of transactions in the blocks:', 5058)


## B. Stock Market Data Analysis

In [8]:
hdfs_folder_path = "hdfs://localhost:/flume/data/"
data_stocks = sc.wholeTextFiles(hdfs_folder_path)
split = data_stocks.flatMap(lambda x: __import__('re').split(r'\r\n|\n', x[1]))
header = split.first()
RDD = split.filter(lambda line: line != header and line != "")

In [9]:
# Count the number of records
total_records = RDD.count()
print("Total number of records:", total_records)

('Total number of records:', 9775)


In [10]:
unique_days = RDD.map(lambda x: x.split(',')[0]).distinct().count()
print("Number of different days in the table:",unique_days)

('Number of different days in the table:', 5)


In [11]:
records_per_day = RDD.map(lambda x: (x.split(',')[0], 1)).reduceByKey(lambda x, y: x + y).collect()
records_per_day

[(u'GOOG', 1955),
 (u'AAPL', 1955),
 (u'AMZN', 1955),
 (u'MSFT', 1955),
 (u'IBM', 1955)]

In [12]:
# Collect distinct symbols
symbols = RDD.map(lambda x: x.split(',')[0]).distinct().collect()
print("Symbols in the table:", symbols)

('Symbols in the table:', [u'GOOG', u'AAPL', u'AMZN', u'MSFT', u'IBM'])


In [13]:
# Group data by symbol and find the highest price within each group
highest_price_per_symbol = RDD.map(lambda x: (x.split(',')[0], float(x.split(',')[2]))).\
    groupByKey().\
    mapValues(max).\
    collect()

# Print the result
print("Highest Price per Symbol:")
for symbol, highest_price in highest_price_per_symbol:
    print(symbol, highest_price)

Highest Price per Symbol:
(u'GOOG', 152.63)
(u'AAPL', 178.66)
(u'AMZN', 181.005)
(u'MSFT', 430.75)
(u'IBM', 193.95)


In [14]:
lowest_price_per_symbol = RDD.map(lambda x: (x.split(',')[0], float(x.split(',')[5]))).reduceByKey(min)
print("Lowest price for each symbol:")
for record in lowest_price_per_symbol.collect():
    print(record)

Lowest price for each symbol:
(u'GOOG', 147.045)
(u'AAPL', 170.075)
(u'AMZN', 173.915)
(u'MSFT', 415.03)
(u'IBM', 190.07)


In [15]:
average_price_per_symbol = RDD.map(lambda x: (x.split(',')[0], (float(x.split(',')[2]), 1))).\
    reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])).\
    map(lambda x: (x[0], x[1][0] / x[1][1])).collect()
average_price_per_symbol

[(u'GOOG', 149.36608951406657),
 (u'AAPL', 174.54984040920726),
 (u'AMZN', 176.86760358056276),
 (u'MSFT', 423.7469294117647),
 (u'IBM', 192.19820306905365)]

In [16]:
price_range_per_symbol = RDD.map(lambda x: (x.split(',')[0], (float(x.split(',')[2]), float(x.split(',')[3])))).\
    reduceByKey(lambda x, y: (min(x[0], y[0]), max(x[1], y[1]))).collect()
price_range_per_symbol

[(u'GOOG', (147.05, 152.93)),
 (u'AAPL', (170.075, 178.68)),
 (u'AMZN', (173.87, 181.415)),
 (u'MSFT', (414.25, 430.82)),
 (u'IBM', (190.24, 194.0))]

In [17]:
date_highest_price_per_symbol = RDD.map(lambda x: (x.split(',')[0], (float(x.split(',')[2]), x.split(',')[1]))).\
    reduceByKey(lambda x, y: x if x[0] > y[0] else y).collect()
date_highest_price_per_symbol

[(u'GOOG', (152.63, u'2024-03-18 09:46:00')),
 (u'AAPL', (178.66, u'2024-03-20 16:00:00')),
 (u'AMZN', (181.005, u'2024-03-21 09:33:00')),
 (u'MSFT', (430.75, u'2024-03-21 09:35:00')),
 (u'IBM', (193.95, u'2024-03-20 16:00:00'))]

## C. Blockchain Data Analysis – Part 2

In [18]:
hdfs_path = "/user/hive/warehouse/project1.db"

blocks_rdd = sc.textFile(hdfs_path + "/blocks_2023_sep_10_to_15")
blocks_info_rdd = sc.textFile(hdfs_path + "/blocks_info_2023_sep_10_to_15")
tx_info_rdd = sc.textFile(hdfs_path + "/tx_info_2023_sep_10_to_15")

In [19]:
total_blocks = blocks_rdd.count()
print("Total number of blocks:", total_blocks)

('Total number of blocks:', 920)


In [20]:
largest_block_height = blocks_rdd.map(lambda x: int(x.split('\x01')[3])).max()

print("Largest block height:", largest_block_height)

('Largest block height:', 807751)


In [ ]:
largest_block_row = blocks_info_rdd.filter(lambda x: int(x.split('\x01')[9]) == largest_block_height).first()
largest_height_hash = largest_block_row.split('\x01')[1]
print("largest_height_hash:", largest_height_hash)

matching_row = blocks_rdd.filter(lambda x: x.split('\x01')[1] == largest_height_hash).first()
timestamp = matching_row.split('\x01')[2]

print("Timestamp for the block with the largest height:", timestamp)

In [22]:
block_hashes = tx_info_rdd.map(lambda row: (row.split('\x01')[2], 1))

block_transaction_counts = block_hashes.groupByKey().mapValues(len)

largest_block = block_transaction_counts.top(1, key=lambda x: x[1])

print("Block with the largest number of transactions:")
print("Block Hash:", largest_block[0][0])
print("Number of Transactions:", largest_block[0][1])

Block with the largest number of transactions:
('Block Hash:', u'00000000000000000002dbf9d0bc1c743ac17bdb60d5c6abc8cd94f2d253621d')
('Number of Transactions:', 7252)


# Step 3: Data Analysis Using Spark SQL

## A. Blockchain Data Analysis – Part 1

In [23]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

blocks_df = sqlContext.createDataFrame(data_blocks)

blocks_df.registerTempTable("blocks")

total_blocks = sqlContext.sql("SELECT COUNT(*) as total_blocks FROM blocks").first().total_blocks
print("Total Blocks: {}".format(total_blocks))

Total Blocks: 1026


/usr/lib/spark/python/pyspark/sql/context.py:156: UserWarning: Using RDD of dict to inferSchema is deprecated,please use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated,"


In [24]:
max_block_height = sqlContext.sql("SELECT MAX(height) as max_block_height FROM blocks").first().max_block_height
print("Largest Block Height: {}".format(max_block_height))

Largest Block Height: 835509


In [25]:
block_info = sqlContext.sql("SELECT time FROM blocks WHERE height = '{}'".format(max_block_height)).first()
timestamp_value = block_info.time
print("Time for the largest block: {}".format(timestamp_value))

Time for the largest block: 1710924274


In [26]:
max_transactions = sqlContext.sql("SELECT MAX(n_tx) as max_transactions FROM blocks").first().max_transactions
print("Highest Number of Transactions: {}".format(max_transactions))

Highest Number of Transactions: 5058


## B. Stock Market Data Analysis

In [27]:
from pyspark.sql import Row
def parse_line(line):
    parts = line.split(',')
    return Row(symbol=parts[0], date=parts[1],open=float(parts[2]), high=float(parts[3]), 
               low=float(parts[4]), close=float(parts[5]), 
               volume=int(parts[6]))

# Convert RDD to DataFrame
df = sqlContext.createDataFrame(RDD.map(parse_line))

# Register the DataFrame as a temp table
df.registerTempTable("stock_data")

result = sqlContext.sql("SELECT * FROM stock_data")
result.show()

close   date                high    low     open    symbol volume  
172.39  2024-03-22 16:00:00 172.521 172.248 172.37  AAPL   17915930
172.25  2024-03-22 15:59:00 172.48  172.23  172.479 AAPL   953094  
172.48  2024-03-22 15:58:00 172.49  172.39  172.4   AAPL   274813  
172.405 2024-03-22 15:57:00 172.42  172.33  172.34  AAPL   342157  
172.35  2024-03-22 15:56:00 172.43  172.33  172.37  AAPL   288187  
172.38  2024-03-22 15:55:00 172.47  172.28  172.29  AAPL   375835  
172.3   2024-03-22 15:54:00 172.535 172.26  172.485 AAPL   390478  
172.49  2024-03-22 15:53:00 172.51  172.41  172.42  AAPL   259557  
172.433 2024-03-22 15:52:00 172.47  172.37  172.39  AAPL   176532  
172.4   2024-03-22 15:51:00 172.51  172.3   172.46  AAPL   363401  
172.465 2024-03-22 15:50:00 172.63  172.415 172.58  AAPL   290657  
172.58  2024-03-22 15:49:00 172.638 172.56  172.635 AAPL   140935  
172.63  2024-03-22 15:48:00 172.68  172.56  172.591 AAPL   142526  
172.595 2024-03-22 15:47:00 172.68  172.59  172.

In [28]:
record_count = sqlContext.sql("SELECT COUNT(*) as record_count FROM stock_data").first().record_count
print("Number of Records: {}".format(record_count))

Number of Records: 9775


In [29]:
distinct_days = sqlContext.sql("SELECT COUNT(DISTINCT SUBSTRING(`date`, 1, 10)) as distinct_days FROM stock_data").first().distinct_days
print("Number of Different Days: {}".format(distinct_days))

Number of Different Days: 5


In [30]:
# Execute the SQL query and collect the result
records_per_day = sqlContext.sql("SELECT SUBSTRING(`date`, 1, 10), COUNT(*) FROM stock_data GROUP BY SUBSTRING(`date`, 1, 10)")
records_per_day.show()


c0         c1  
2024-03-18 1955
2024-03-19 1955
2024-03-20 1955
2024-03-21 1955
2024-03-22 1955


In [31]:
symbols = sqlContext.sql("SELECT DISTINCT `symbol` as symbol FROM stock_data")
symbols.show()

symbol
AAPL  
IBM   
AMZN  
MSFT  
GOOG  


In [32]:
highest_price = sqlContext.sql("SELECT `symbol` as symbol, MAX(`high`) as highest_price FROM stock_data GROUP BY `symbol`")
highest_price.show()

symbol highest_price
AAPL   178.68       
IBM    194.0        
AMZN   181.415      
MSFT   430.82       
GOOG   152.93       


In [33]:
lowest_price = sqlContext.sql("SELECT `symbol` as symbol, MIN(`low`) as lowest_price FROM stock_data GROUP BY `symbol`")
lowest_price.show()

symbol lowest_price
AAPL   170.06      
IBM    190.01      
AMZN   173.52      
MSFT   413.78      
GOOG   147.01      


In [34]:
average_price = sqlContext.sql("SELECT `symbol` as symbol, AVG(`close`) as average_price FROM stock_data GROUP BY `symbol`")
average_price.show()

symbol average_price     
AAPL   174.54806649616415
IBM    192.19785984654712
AMZN   176.86847007672645
MSFT   423.75076368286466
GOOG   149.36463836317134


In [35]:
price_range = sqlContext.sql("SELECT `symbol` as symbol, MIN(`close`) as Minimum, MAX(`close`) as Maximum, MAX(`close`) - MIN(`close`) as price_range FROM stock_data GROUP BY `symbol`")
price_range.show()

symbol Minimum Maximum price_range       
AAPL   170.075 178.65  8.575000000000017 
IBM    190.07  193.97  3.9000000000000057
AMZN   173.915 180.995 7.0800000000000125
MSFT   415.03  430.71  15.680000000000007
GOOG   147.045 152.65  5.605000000000018 


In [36]:
date_highest_price = sqlContext.sql("""
SELECT s.`symbol` as symbol, s.`date` as transaction_date, s.`high` as highest_price
FROM stock_data s
JOIN (
    SELECT `symbol` as symbol, MAX(`high`) as max_price
    FROM stock_data
    GROUP BY `symbol`
) t
ON s.`symbol` = t.symbol AND s.`high` = t.max_price
""")
date_highest_price.show()

symbol transaction_date    highest_price
IBM    2024-03-20 16:00:00 194.0        
AMZN   2024-03-21 09:33:00 181.415      
MSFT   2024-03-21 09:35:00 430.82       
GOOG   2024-03-18 09:46:00 152.93       
AAPL   2024-03-20 16:00:00 178.68       


## C. Blockchain Data Analysis – Part 2

In [39]:
blocks = sqlCtx.load(source="jdbc", 
                         url="jdbc:mysql://localhost/project1?user=training&password=training", 
                         dbtable="blocks_2023_Sep_10_to_15")

In [38]:
blocks_info = sqlCtx.load(source="jdbc", 
                         url="jdbc:mysql://localhost/project1?user=training&password=training", 
                         dbtable="blocks_info_2023_Sep_10_to_15")

tx_info = sqlCtx.load(source="jdbc", 
                         url="jdbc:mysql://localhost/project1?user=training&password=training", 
                         dbtable="tx_info_2023_Sep_10_to_15")

In [40]:
total_blocks = blocks.count()
print("Total number of blocks: {}".format(total_blocks))

Total number of blocks: 920


In [41]:
largest_block_height = blocks.agg({"block_index": "max"}).collect()[0][0]
print("Largest block height:", largest_block_height)

('Largest block height:', 807751)


In [42]:
largest_block_date_time = blocks.filter(blocks["block_index"] == largest_block_height).select("time").collect()[0][0]
formatted_time = largest_block_date_time.strftime("%Y-%m-%d %H:%M:%S")

print("Date and time for the largest block:", formatted_time)

('Date and time for the largest block:', '2023-09-15 00:00:00')


In [43]:
largest_num_transactions = tx_info.groupBy("block_index").count().agg({"count": "max"}).collect()[0][0]
print("Largest number of transactions in a block:", largest_num_transactions)

('Largest number of transactions in a block:', 7252)
